In [3]:
#!/usr/bin/env python
# coding: utf-8

"""
NFL Game Prediction and Team Rankings Analysis
===============================================
This script predicts NFL game outcomes using historical data (2022-2023)
and evaluates performance on 2024 season data using multiple ML models:
- Logistic Regression
- K-Nearest Neighbors (KNN)
- Random Forest
- Stacked Ensemble

It generates team rankings, enriches them with salary and standings data,
and produces comprehensive visualizations and correlation analyses.

Author: Data Analysis Team
Date: 2024
"""

import warnings
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from scipy.stats import spearmanr, pearsonr
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    classification_report
)
from sklearn.inspection import permutation_importance
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

warnings.filterwarnings('ignore')
sns.set(style="whitegrid", context="talk")


# ==============================================================================
# CONSTANTS AND CONFIGURATION
# ==============================================================================

# Detectar ubicación del notebook
import os

if '__file__' in globals():
    # Si corre como script .py
    NOTEBOOK_DIR = Path(__file__).parent
else:
    # Si corre en Jupyter
    NOTEBOOK_DIR = Path(os.getcwd())
    # Ajustar si getcwd() no es la carpeta notebook/
    if NOTEBOOK_DIR.name != 'notebook':
        NOTEBOOK_DIR = NOTEBOOK_DIR / 'notebook'

# Subir a AdvProgramming_Final/
PROJECT_ROOT = NOTEBOOK_DIR.parent
DATA_DIR = PROJECT_ROOT / "data"
OUTPUT_DIR = PROJECT_ROOT / "data"

print(f"📂 Proyecto: {PROJECT_ROOT}")
print(f"📊 Datos: {DATA_DIR}")
print(f"💾 Output: {OUTPUT_DIR}")

TEAM_NAME_MAP = {
    'KC Chiefs': 'KC', 'ATL Falcons': 'ATL', 'BAL Ravens': 'BAL',
    'LAR Rams': 'LA', 'CHI Bears': 'CHI', 'IND Colts': 'IND',
    'WAS Command': 'WAS', 'CIN Bengals': 'CIN', 'LAC Chargers': 'LAC',
    'PIT Steelers': 'PIT', 'HOU Texans': 'HOU', 'BUF Bills': 'BUF',
    'SEA Seahawks': 'SEA', 'PHI Eagles': 'PHI', 'ARI Cardinals': 'ARI',
    'MIN Vikings': 'MIN', 'NYJ Jets': 'NYJ', 'GB Packers': 'GB',
    'DET Lions': 'DET', 'DEN Broncos': 'DEN', 'CAR Panthers': 'CAR',
    'NO Saints': 'NO', 'JAX Jaguars': 'JAX', 'TEN Titans': 'TEN',
    'TB Buccaneers': 'TB', 'MIA Dolphins': 'MIA', 'SF 49ers': 'SF',
    'CLE Browns': 'CLE', 'NE Patriots': 'NE', 'NYG Giants': 'NYG',
    'LV Raiders': 'LV'
}

STAT_COLS = [
    'passing_yards', 'passing_tds', 'passing_interceptions', 'passing_epa',
    'rushing_yards', 'rushing_tds', 'rushing_epa',
    'receiving_yards', 'receiving_tds', 'receiving_epa',
    'def_sacks', 'def_interceptions', 'def_tds',
    'turnovers', 'penalties', 'penalty_yards'
]


# ==============================================================================
# DATA LOADING AND PREPROCESSING
# ==============================================================================

def load_and_prepare_data(dict_path, games_path):
    """
    Load NFL data dictionary and games data, then apply type conversions.
    
    Parameters
    ----------
    dict_path : Path
        Path to the data dictionary Excel file
    games_path : Path
        Path to the games Excel file
        
    Returns
    -------
    pd.DataFrame
        Preprocessed games DataFrame with correct column types
    """
    print("=" * 60)
    print("LOADING DATA")
    print("=" * 60)
    
    dict_df = pd.read_excel(dict_path)
    games_df = pd.read_excel(games_path)
    
    # Normalize column names
    dict_df.columns = dict_df.columns.str.lower().str.strip()
    games_df.columns = games_df.columns.str.lower().str.strip()
    
    # Build type mapping
    type_map = dict(zip(dict_df['field'], dict_df['type']))
    
    # Convert types
    for col, dtype in type_map.items():
        if col not in games_df.columns:
            continue
        if "numeric" in dtype.lower():
            games_df[col] = pd.to_numeric(games_df[col], errors='coerce')
        elif "character" in dtype.lower() or "categorical" in dtype.lower():
            games_df[col] = games_df[col].astype("string")
    
    print(f"Columns: {len(games_df.columns)} | Rows: {len(games_df)}")
    print("Seasons available:", games_df['season'].unique())
    print("Season types:", games_df['season_type'].unique())
    
    return games_df


def create_home_away_perspective(games_df):
    """
    Expand games data to create both home and away team perspectives.
    Each game becomes two rows, one from each team's viewpoint.
    
    Parameters
    ----------
    games_df : pd.DataFrame
        Original games data
        
    Returns
    -------
    pd.DataFrame
        Expanded DataFrame with team, opponent, points_for, points_against
    """
    print("\n" + "=" * 60)
    print("CREATING TEAM PERSPECTIVES")
    print("=" * 60)
    
    # Home perspective
    home_games = games_df.copy()
    home_games['team'] = home_games['home_team']
    home_games['opponent_team'] = home_games['away_team']
    home_games['points_for'] = home_games['home_score']
    home_games['points_against'] = home_games['away_score']
    
    # Away perspective
    away_games = games_df.copy()
    away_games['team'] = away_games['away_team']
    away_games['opponent_team'] = away_games['home_team']
    away_games['points_for'] = away_games['away_score']
    away_games['points_against'] = away_games['home_score']
    
    # Combine
    combined_df = pd.concat([home_games, away_games], ignore_index=True)
    combined_df['win'] = (
        combined_df['points_for'] > combined_df['points_against']
    ).astype(int)
    
    # Create turnovers feature
    combined_df['turnovers'] = (
        combined_df['passing_interceptions'].fillna(0) +
        combined_df['sack_fumbles_lost'].fillna(0) +
        combined_df['rushing_fumbles_lost'].fillna(0) +
        combined_df['receiving_fumbles_lost'].fillna(0)
    )
    
    print(f"Dataset expanded to {len(combined_df)} rows (2 per game)")
    print(f"Average turnovers per game: {combined_df['turnovers'].mean():.2f}")
    
    return combined_df


def create_team_season_stats(df, stat_cols):
    """
    Create rolling historical averages for each team within each season.
    Uses expanding window with shift to avoid data leakage.
    
    Parameters
    ----------
    df : pd.DataFrame
        Games data with team perspective
    stat_cols : list
        List of statistical columns to aggregate
        
    Returns
    -------
    pd.DataFrame
        DataFrame with added historical average columns
    """
    print("\n" + "=" * 60)
    print("CREATING HISTORICAL FEATURES")
    print("=" * 60)
    
    df = df.sort_values(['team', 'season', 'week']).reset_index(drop=True)
    
    # Create rolling averages for each stat
    for col in stat_cols:
        df[f'{col}_team_avg'] = (
            df.groupby(['team', 'season'])[col]
            .expanding()
            .mean()
            .reset_index(level=[0, 1], drop=True)
            .shift(1)
        )
    
    # Create win percentage
    df['win_pct'] = (
        df.groupby(['team', 'season'])['win']
        .expanding()
        .mean()
        .reset_index(level=[0, 1], drop=True)
        .shift(1)
    )
    
    # Fill NaN values
    df['win_pct'] = df['win_pct'].fillna(0.5)
    for col in stat_cols:
        avg_col = f'{col}_team_avg'
        df[avg_col] = df[avg_col].fillna(df[avg_col].median())
    
    print(f"Created {len(stat_cols)} historical average features")
    
    return df


def add_opponent_features(df):
    """
    Add opponent's historical statistics to each game record.
    
    Parameters
    ----------
    df : pd.DataFrame
        DataFrame with team historical features
        
    Returns
    -------
    pd.DataFrame
        DataFrame with added opponent features
    """
    print("\n" + "=" * 60)
    print("ADDING OPPONENT FEATURES")
    print("=" * 60)
    
    feature_cols = [c for c in df.columns if '_team_avg' in c or c == 'win_pct']
    print(f"Merging {len(feature_cols)} opponent features...")
    
    # Create opponent dataframe
    opp = df[['season', 'week', 'team'] + feature_cols].copy()
    rename_dict = {'team': 'opponent_team'}
    rename_dict.update({col: f'opp_{col}' for col in feature_cols})
    opp = opp.rename(columns=rename_dict)
    
    # Merge opponent features
    df = df.merge(opp, on=['season', 'week', 'opponent_team'], how='left')
    
    # Fill missing opponent features with median
    for col in feature_cols:
        opp_col = f'opp_{col}'
        if opp_col in df.columns:
            df[opp_col] = df[opp_col].fillna(df[col].median())
    
    return df


def create_relative_features(df):
    """
    Create relative features (team vs opponent differences).
    
    Parameters
    ----------
    df : pd.DataFrame
        DataFrame with team and opponent features
        
    Returns
    -------
    pd.DataFrame
        DataFrame with added relative features
    """
    print("\n" + "=" * 60)
    print("CREATING RELATIVE FEATURES")
    print("=" * 60)
    
    count = 0
    for col in [c for c in df.columns if '_team_avg' in c]:
        opp_col = f"opp_{col}"
        if opp_col in df.columns:
            df[f'rel_{col}'] = df[col] - df[opp_col]
            count += 1
    
    df['is_home'] = (df['team'] == df['home_team']).astype(int)
    df['is_playoff'] = (df['season_type'] == 'POST').astype(int)
    
    print(f"Created {count} relative features plus is_home and is_playoff")
    
    return df


def split_train_test(df, train_seasons, test_season):
    """
    Split data into training and test sets by season.
    
    Parameters
    ----------
    df : pd.DataFrame
        Full games DataFrame
    train_seasons : list
        List of seasons for training
    test_season : str
        Season for testing
        
    Returns
    -------
    tuple
        (train_df, test_df)
    """
    print("\n" + "=" * 60)
    print("SPLITTING TRAIN/TEST DATA")
    print("=" * 60)
    
    df['season'] = df['season'].astype(str).str.strip()
    
    train_df = df[
        (df['season'].isin(train_seasons)) & 
        (df['season_type'] == 'REG')
    ].copy()
    
    test_df = df[
        (df['season'] == test_season) & 
        (df['season_type'] == 'REG')
    ].copy()
    
    print(f"Training samples: {len(train_df)} ({', '.join(train_seasons)})")
    print(f"Test samples: {len(test_df)} ({test_season})")
    
    return train_df, test_df


# ==============================================================================
# MODEL TRAINING AND EVALUATION
# ==============================================================================

def prepare_features(train_df, test_df):
    """
    Extract feature columns and prepare X, y for training and testing.
    
    Parameters
    ----------
    train_df : pd.DataFrame
        Training data
    test_df : pd.DataFrame
        Test data
        
    Returns
    -------
    tuple
        (X_train, y_train, X_test, y_test, feature_cols, scaler)
    """
    print("\n" + "=" * 60)
    print("PREPARING FEATURES")
    print("=" * 60)
    
    feature_cols = (
        [c for c in train_df.columns if c.startswith('rel_')] +
        ['win_pct', 'opp_win_pct', 'is_home', 'is_playoff']
    )
    feature_cols = [c for c in feature_cols if c in train_df.columns]
    
    X_train = train_df[feature_cols].fillna(0)
    y_train = train_df['win']
    X_test = test_df[feature_cols].fillna(0)
    y_test = test_df['win']
    
    # Scale features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    print(f"Features used: {len(feature_cols)}")
    print("Sample features:", feature_cols[:5])
    
    return X_train_scaled, y_train, X_test_scaled, y_test, feature_cols, scaler


def train_logistic_regression(X_train, y_train, X_test, y_test):
    """
    Train and evaluate Logistic Regression model.
    
    Parameters
    ----------
    X_train : np.ndarray
        Training features
    y_train : pd.Series
        Training labels
    X_test : np.ndarray
        Test features
    y_test : pd.Series
        Test labels
        
    Returns
    -------
    tuple
        (model, y_pred, y_proba, accuracy)
    """
    print("\n" + "=" * 60)
    print("TRAINING LOGISTIC REGRESSION")
    print("=" * 60)
    
    model = LogisticRegression(max_iter=1000, random_state=42)
    model.fit(X_train, y_train)
    
    cv_scores = cross_val_score(
        model, X_train, y_train, cv=5, scoring='accuracy'
    )
    print(f"Cross-validation accuracy: {cv_scores.mean():.3f} "
          f"(+/- {cv_scores.std():.3f})")
    
    y_pred = model.predict(X_test)
    y_proba = model.predict_proba(X_test)[:, 1]
    accuracy = accuracy_score(y_test, y_pred)
    
    print(f"Test accuracy: {accuracy:.3f}")
    print("\nClassification Report:")
    print(classification_report(y_test, y_pred))
    
    return model, y_pred, y_proba, accuracy


def train_knn(X_train, y_train, X_test, y_test):
    """
    Train and evaluate K-Nearest Neighbors model with GridSearchCV.
    
    Parameters
    ----------
    X_train : np.ndarray
        Training features
    y_train : pd.Series
        Training labels
    X_test : np.ndarray
        Test features
    y_test : pd.Series
        Test labels
        
    Returns
    -------
    tuple
        (best_model, y_pred, y_proba, accuracy, best_params)
    """
    print("\n" + "=" * 60)
    print("TRAINING KNN WITH GRID SEARCH")
    print("=" * 60)
    
    param_grid = {
        'n_neighbors': [3, 5, 7, 9, 11],
        'weights': ['uniform', 'distance'],
        'p': [1, 2]
    }
    
    knn = KNeighborsClassifier()
    grid_search = GridSearchCV(
        knn, param_grid, cv=5, scoring='accuracy', n_jobs=-1, verbose=0
    )
    grid_search.fit(X_train, y_train)
    
    best_model = grid_search.best_estimator_
    y_pred = best_model.predict(X_test)
    y_proba = best_model.predict_proba(X_test)[:, 1]
    accuracy = accuracy_score(y_test, y_pred)
    
    print(f"Best parameters: {grid_search.best_params_}")
    print(f"Test accuracy: {accuracy:.3f}")
    print("\nClassification Report:")
    print(classification_report(y_test, y_pred))
    
    return best_model, y_pred, y_proba, accuracy, grid_search.best_params_


def train_random_forest(X_train, y_train, X_test, y_test):
    """
    Train and evaluate Random Forest model with GridSearchCV.
    
    Parameters
    ----------
    X_train : np.ndarray
        Training features
    y_train : pd.Series
        Training labels
    X_test : np.ndarray
        Test features
    y_test : pd.Series
        Test labels
        
    Returns
    -------
    tuple
        (best_model, y_pred, y_proba, accuracy, best_params)
    """
    print("\n" + "=" * 60)
    print("TRAINING RANDOM FOREST WITH GRID SEARCH")
    print("=" * 60)
    
    param_grid = {
        'n_estimators': [100, 300],
        'max_depth': [5, 10, None],
        'min_samples_leaf': [1, 2, 4]
    }
    
    rf = RandomForestClassifier(random_state=42, n_jobs=-1)
    grid_search = GridSearchCV(
        rf, param_grid, cv=4, scoring='accuracy', n_jobs=-1, verbose=0
    )
    grid_search.fit(X_train, y_train)
    
    best_model = grid_search.best_estimator_
    y_pred = best_model.predict(X_test)
    y_proba = best_model.predict_proba(X_test)[:, 1]
    accuracy = accuracy_score(y_test, y_pred)
    
    print(f"Best parameters: {grid_search.best_params_}")
    print(f"Test accuracy: {accuracy:.3f}")
    print("\nClassification Report:")
    print(classification_report(y_test, y_pred))
    
    return best_model, y_pred, y_proba, accuracy, grid_search.best_params_


def train_stacked_model(models_dict, X_train, y_train, X_test, y_test):
    """
    Train a stacked ensemble model using base model predictions.
    
    Parameters
    ----------
    models_dict : dict
        Dictionary of trained models {'lr': model, 'knn': model, 'rf': model}
    X_train : np.ndarray
        Training features
    y_train : pd.Series
        Training labels
    X_test : np.ndarray
        Test features
    y_test : pd.Series
        Test labels
        
    Returns
    -------
    tuple
        (stacker, y_pred, y_proba, accuracy)
    """
    print("\n" + "=" * 60)
    print("TRAINING STACKED ENSEMBLE MODEL")
    print("=" * 60)
    
    # Generate meta-features from base models
    train_meta = pd.DataFrame({
        'lr_prob': models_dict['lr'].predict_proba(X_train)[:, 1],
        'knn_prob': models_dict['knn'].predict_proba(X_train)[:, 1],
        'rf_prob': models_dict['rf'].predict_proba(X_train)[:, 1]
    })
    
    test_meta = pd.DataFrame({
        'lr_prob': models_dict['lr'].predict_proba(X_test)[:, 1],
        'knn_prob': models_dict['knn'].predict_proba(X_test)[:, 1],
        'rf_prob': models_dict['rf'].predict_proba(X_test)[:, 1]
    })
    
    # Train meta-model
    stacker = LogisticRegression(max_iter=1000, random_state=42)
    stacker.fit(train_meta, y_train)
    
    # Evaluate
    y_pred = stacker.predict(test_meta)
    y_proba = stacker.predict_proba(test_meta)[:, 1]
    accuracy = accuracy_score(y_test, y_pred)
    
    print(f"Stacked model accuracy: {accuracy:.3f}")
    print("\nClassification Report:")
    print(classification_report(y_test, y_pred))
    
    # Show meta-model coefficients
    coef_df = pd.DataFrame({
        'base_model': ['lr_prob', 'knn_prob', 'rf_prob'],
        'coefficient': stacker.coef_[0]
    }).sort_values('coefficient', key=abs, ascending=False)
    
    print("\nMeta-Model Coefficients:")
    print(coef_df.to_string(index=False))
    
    return stacker, y_pred, y_proba, accuracy


# ==============================================================================
# ANALYSIS AND RANKINGS
# ==============================================================================

def compute_permutation_importance(model, X_test, y_test, feature_cols, 
                                   model_name):
    """
    Compute permutation importance for a model.
    
    Parameters
    ----------
    model : sklearn model
        Trained model
    X_test : np.ndarray
        Test features
    y_test : pd.Series
        Test labels
    feature_cols : list
        List of feature names
    model_name : str
        Name of the model for display
        
    Returns
    -------
    pd.DataFrame
        DataFrame with feature importances
    """
    print(f"\nComputing permutation importance for {model_name}...")
    
    perm = permutation_importance(
        model, X_test, y_test, n_repeats=10, random_state=42, n_jobs=-1
    )
    
    importance_df = pd.DataFrame({
        'feature': feature_cols,
        'perm_importance_mean': perm.importances_mean,
        'perm_importance_std': perm.importances_std
    }).sort_values('perm_importance_mean', ascending=False)
    
    return importance_df


def compute_team_rankings(test_df, y_proba, model_name):
    """
    Compute team rankings based on model predictions.
    
    Parameters
    ----------
    test_df : pd.DataFrame
        Test data with team information
    y_proba : np.ndarray
        Predicted win probabilities
    model_name : str
        Name of the model
        
    Returns
    -------
    pd.DataFrame
        DataFrame with team rankings
    """
    pred_df = test_df[['team', 'win']].copy()
    pred_df['predicted_prob'] = y_proba
    
    # Team strength (average predicted win probability)
    team_strength = (
        pred_df.groupby('team')['predicted_prob']
        .mean()
        .reset_index()
        .rename(columns={'predicted_prob': 'avg_pred_prob'})
    )
    
    # Actual win rate
    actual_win_rate = (
        pred_df.groupby('team')['win']
        .mean()
        .reset_index()
        .rename(columns={'win': 'actual_win_pct'})
    )
    
    # Combine and rank
    rankings = team_strength.merge(actual_win_rate, on='team', how='left')
    rankings = rankings.sort_values(
        'avg_pred_prob', ascending=False
    ).reset_index(drop=True)
    rankings['predicted_rank'] = rankings.index + 1
    rankings['actual_rank'] = rankings['actual_win_pct'].rank(ascending=False)
    rankings['rank_diff'] = rankings['predicted_rank'] - rankings['actual_rank']
    rankings['model'] = model_name
    
    # Compute correlation
    corr = rankings['avg_pred_prob'].corr(rankings['actual_win_pct'])
    print(f"\n{model_name} - Correlation between predicted and actual: {corr:.3f}")
    
    return rankings


def enrich_rankings_with_external_data(rankings_df, salary_df, 
                                       standings_df, team_map):
    """
    Enrich rankings with salary and standings data.
    
    Parameters
    ----------
    rankings_df : pd.DataFrame
        Team rankings DataFrame
    salary_df : pd.DataFrame
        Salary data by team
    standings_df : pd.DataFrame
        Official standings data
    team_map : dict
        Mapping of team names to abbreviations
        
    Returns
    -------
    pd.DataFrame
        Enriched rankings DataFrame
    """
    print("\n" + "=" * 60)
    print("ENRICHING RANKINGS WITH EXTERNAL DATA")
    print("=" * 60)
    
    # Map team names to abbreviations
    salary_df['team_abbrev'] = salary_df['team'].map(team_map)
    
    # Merge salary data
    salary_cols = [
        'team_abbrev', 'median_salary', 'avg_salary',
        'total_salary', 'num_players'
    ]
    enriched = rankings_df.merge(
        salary_df[salary_cols],
        left_on='team',
        right_on='team_abbrev',
        how='left'
    ).drop(columns=['team_abbrev'])
    
    # Merge standings data
    standings_cols = [
        'team', 'wins', 'losses', 'win_pct', 'points_scored',
        'points_allowed', 'point_diff', 'games_played'
    ]
    enriched = enriched.merge(
        standings_df[standings_cols],
        on='team',
        how='left'
    )
    
    print("Enrichment complete")
    
    return enriched


# ==============================================================================
# VISUALIZATION FUNCTIONS
# ==============================================================================

def plot_confusion_matrices(results_dict, y_test):
    """
    Plot side-by-side confusion matrices for all models.
    
    Parameters
    ----------
    results_dict : dict
        Dictionary of model results {'model_name': {'y_pred': ...}, ...}
    y_test : pd.Series
        True test labels
    """
    print("\n" + "=" * 60)
    print("GENERATING CONFUSION MATRICES")
    print("=" * 60)
    
    fig, axes = plt.subplots(1, len(results_dict), figsize=(4*len(results_dict), 4))
    
    if len(results_dict) == 1:
        axes = [axes]
    
    for ax, (model_name, results) in zip(axes, results_dict.items()):
        cm = confusion_matrix(y_test, results['y_pred'], labels=[0, 1])
        sns.heatmap(
            cm, annot=True, fmt="d", cmap="Blues", cbar=False, ax=ax,
            xticklabels=["Pred Loss", "Pred Win"],
            yticklabels=["True Loss", "True Win"]
        )
        ax.set_ylabel("Actual")
        ax.set_xlabel("Predicted")
        ax.set_title(f"{model_name} Confusion Matrix")
    
    plt.tight_layout()
    plt.show()


def plot_predicted_vs_actual(enriched_rankings):
    """
    Plot predicted vs actual win percentage for each model.
    
    Parameters
    ----------
    enriched_rankings : pd.DataFrame
        Enriched rankings with all models
    """
    print("\n" + "=" * 60)
    print("GENERATING PREDICTED VS ACTUAL PLOTS")
    print("=" * 60)
    
    models = enriched_rankings['model'].unique()
    
    for model_name in models:
        df_model = enriched_rankings[enriched_rankings['model'] == model_name]
        
        plt.figure(figsize=(8, 6))
        sns.scatterplot(
            data=df_model, x='avg_pred_prob', y='actual_win_pct',
            hue='team', s=100, alpha=0.7
        )
        sns.regplot(
            data=df_model, x='avg_pred_prob', y='actual_win_pct',
            scatter=False, color='black', line_kws={'linewidth': 2}
        )
        
        # Add correlation text
        r, _ = pearsonr(df_model['avg_pred_prob'], df_model['actual_win_pct'])
        plt.text(
            0.05, 0.95, f"Pearson r = {r:.3f}",
            transform=plt.gca().transAxes,
            fontsize=12, verticalalignment='top',
            bbox=dict(boxstyle="round", fc="white", ec="gray")
        )
        
        plt.title(f"{model_name}: Predicted vs Actual Win %")
        plt.xlabel("Predicted Win %")
        plt.ylabel("Actual Win %")
        plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
        plt.tight_layout()
        plt.show()


def plot_salary_analysis(enriched_rankings):
    """
    Plot salary vs win percentage analysis.
    
    Parameters
    ----------
    enriched_rankings : pd.DataFrame
        Enriched rankings with salary data
    """
    print("\n" + "=" * 60)
    print("GENERATING SALARY ANALYSIS PLOTS")
    print("=" * 60)
    
    clean_df = enriched_rankings.dropna(
        subset=['median_salary', 'avg_pred_prob', 'actual_win_pct']
    )
    
    models = clean_df['model'].unique()
    
    # Predicted Win % vs Median Salary
    for model_name in models:
        df_model = clean_df[clean_df['model'] == model_name]
        
        plt.figure(figsize=(8, 6))
        sns.scatterplot(
            data=df_model, x='median_salary', y='avg_pred_prob',
            hue='team', s=100, alpha=0.7
        )
        sns.regplot(
            data=df_model, x='median_salary', y='avg_pred_prob',
            scatter=False, color='black', line_kws={'linewidth': 2}
        )
        
        r, _ = pearsonr(df_model['median_salary'], df_model['avg_pred_prob'])
        plt.text(
            0.05, 0.95, f"Pearson r = {r:.3f}",
            transform=plt.gca().transAxes,
            fontsize=12, verticalalignment='top',
            bbox=dict(boxstyle="round", fc="white", ec="gray")
        )
        
        min_sal = df_model['median_salary'].min()
        max_sal = df_model['median_salary'].max()
        plt.text(
            0.05, 0.85,
            f"Salary Range: ${min_sal:,.0f} - ${max_sal:,.0f}",
            transform=plt.gca().transAxes, fontsize=11,
            bbox=dict(boxstyle="round", fc="white", ec="gray")
        )
        
        plt.title(f"{model_name}: Predicted Win % vs Median Salary")
        plt.xlabel("Median Salary ($)")
        plt.ylabel("Predicted Win %")
        plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
        plt.tight_layout()
        plt.show()
    
    # Actual Win % vs Median Salary (all teams)
    plt.figure(figsize=(8, 6))
    sns.scatterplot(
        data=clean_df, x='median_salary', y='actual_win_pct',
        hue='team', s=100, alpha=0.7
    )
    sns.regplot(
        data=clean_df, x='median_salary', y='actual_win_pct',
        scatter=False, color='black', line_kws={'linewidth': 2}
    )
    
    r, _ = pearsonr(clean_df['median_salary'], clean_df['actual_win_pct'])
    plt.text(
        0.05, 0.95, f"Pearson r = {r:.3f}",
        transform=plt.gca().transAxes,
        fontsize=12, verticalalignment='top',
        bbox=dict(boxstyle="round", fc="white", ec="gray")
    )
    
    min_sal = clean_df['median_salary'].min()
    max_sal = clean_df['median_salary'].max()
    plt.text(
        0.05, 0.85,
        f"Salary Range: ${min_sal:,.0f} - ${max_sal:,.0f}",
        transform=plt.gca().transAxes, fontsize=11,
        bbox=dict(boxstyle="round", fc="white", ec="gray")
    )
    
    plt.title("Actual Win % vs Median Salary")
    plt.xlabel("Median Salary ($)")
    plt.ylabel("Actual Win %")
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.tight_layout()
    plt.show()


def plot_rankings_position_bars(position_table, models):
    """
    Plot bar charts comparing predicted vs actual rankings for each model.
    
    Parameters
    ----------
    position_table : pd.DataFrame
        Table with Position, Actual, and model columns
    models : list
        List of model names to plot
    """
    print("\n" + "=" * 60)
    print("GENERATING RANKING POSITION BAR CHARTS")
    print("=" * 60)
    
    actual_order = position_table['Actual'].tolist()
    x = np.arange(len(actual_order))
    
    for model_name in models:
        predicted_order = position_table[model_name].tolist()
        predicted_rank_map = {
            team: rank for rank, team in enumerate(predicted_order, start=1)
        }
        
        actual_ranks = position_table['Position'].tolist()
        predicted_ranks = [
            predicted_rank_map.get(team, len(actual_order) + 1)
            for team in actual_order
        ]
        
        plt.figure(figsize=(14, 6))
        bar_width = 0.6
        
        # Actual ranks (solid)
        plt.bar(
            x, actual_ranks, width=bar_width, color='orange',
            label='Actual Rank'
        )
        # Predicted ranks (transparent)
        plt.bar(
            x, predicted_ranks, width=bar_width, color='steelblue',
            alpha=0.5, label=f'{model_name} Predicted Rank'
        )
        
        plt.xticks(x, actual_order, rotation=45, ha='right')
        plt.xlabel('Teams (ordered by actual standings)')
        plt.ylabel('Rank Position')
        plt.title(f'{model_name}: Predicted vs Actual Rank')
        plt.legend()
        plt.tight_layout()
        plt.show()


def print_correlation_analysis(enriched_rankings):
    """
    Print comprehensive correlation analysis.
    
    Parameters
    ----------
    enriched_rankings : pd.DataFrame
        Enriched rankings DataFrame
    """
    print("\n" + "=" * 60)
    print("CORRELATION ANALYSIS")
    print("=" * 60)
    
    corr_vars = [
        'median_salary', 'avg_salary', 'total_salary',
        'actual_win_pct', 'avg_pred_prob', 'point_diff'
    ]
    
    clean_df = enriched_rankings.dropna(subset=corr_vars)
    
    print("\nPearson Correlation Matrix:")
    print(clean_df[corr_vars].corr(method='pearson').round(3))
    
    print("\nSpearman Correlation Matrix:")
    print(clean_df[corr_vars].corr(method='spearman').round(3))


def create_position_table(rankings_dict, standings_df):
    """
    Create a position table comparing all model rankings.
    
    Parameters
    ----------
    rankings_dict : dict
        Dictionary of rankings DataFrames by model name
    standings_df : pd.DataFrame
        Official standings data
        
    Returns
    -------
    pd.DataFrame
        Position table with actual and predicted rankings
    """
    print("\n" + "=" * 60)
    print("CREATING POSITION TABLE")
    print("=" * 60)
    
    # Sort by actual standings
    actual_order = standings_df.sort_values(
        'win_pct', ascending=False
    )['team'].tolist()
    
    position_data = {'Position': range(1, len(actual_order) + 1)}
    position_data['Actual'] = actual_order
    
    # Add each model's predicted order
    for model_name, rankings_df in rankings_dict.items():
        model_order = rankings_df.sort_values(
            'predicted_rank'
        )['team'].tolist()
        position_data[model_name] = model_order
    
    position_table = pd.DataFrame(position_data)
    
    print("\nTeam Rankings by Position:")
    print(position_table.to_string(index=False))
    
    return position_table


# ==============================================================================
# MAIN EXECUTION PIPELINE
# ==============================================================================

def main():
    """
    Main execution pipeline for NFL prediction analysis.
    """
    print("\n" + "=" * 60)
    print("NFL GAME PREDICTION AND TEAM RANKINGS ANALYSIS")
    print("=" * 60)
    
    # -------------------------------------------------------------------------
    # 1. LOAD AND PREPARE DATA
    # -------------------------------------------------------------------------
    
    games_df = load_and_prepare_data(
        DATA_DIR / "nfl_dictionaries_2.xlsx",
        DATA_DIR / "global_team_games_2.xlsx"
    )
    
    games_df = create_home_away_perspective(games_df)
    games_df = create_team_season_stats(games_df, STAT_COLS)
    games_df = add_opponent_features(games_df)
    games_df = create_relative_features(games_df)
    
    train_df, test_df = split_train_test(
        games_df,
        train_seasons=['2022', '2023'],
        test_season='2024'
    )
    
    # -------------------------------------------------------------------------
    # 2. PREPARE FEATURES AND TRAIN BASE MODELS
    # -------------------------------------------------------------------------
    
    X_train, y_train, X_test, y_test, feature_cols, scaler = prepare_features(
        train_df, test_df
    )
    
    # Train Logistic Regression
    lr_model, lr_pred, lr_proba, lr_acc = train_logistic_regression(
        X_train, y_train, X_test, y_test
    )
    
    # Train KNN
    knn_model, knn_pred, knn_proba, knn_acc, knn_params = train_knn(
        X_train, y_train, X_test, y_test
    )
    
    # Train Random Forest
    rf_model, rf_pred, rf_proba, rf_acc, rf_params = train_random_forest(
        X_train, y_train, X_test, y_test
    )
    
    # -------------------------------------------------------------------------
    # 3. TRAIN STACKED ENSEMBLE MODEL
    # -------------------------------------------------------------------------
    
    models_dict = {'lr': lr_model, 'knn': knn_model, 'rf': rf_model}
    stacker, stack_pred, stack_proba, stack_acc = train_stacked_model(
        models_dict, X_train, y_train, X_test, y_test
    )
    
    # -------------------------------------------------------------------------
    # 4. COMPUTE RANKINGS FOR ALL MODELS
    # -------------------------------------------------------------------------
    
    rank_lr = compute_team_rankings(test_df, lr_proba, "Logistic")
    rank_knn = compute_team_rankings(test_df, knn_proba, "KNN")
    rank_rf = compute_team_rankings(test_df, rf_proba, "RandomForest")
    rank_stack = compute_team_rankings(test_df, stack_proba, "Stacked")
    
    # -------------------------------------------------------------------------
    # 5. ENRICH RANKINGS WITH EXTERNAL DATA
    # -------------------------------------------------------------------------
    
    # Load external data
    salary_df = pd.read_csv(
        DATA_DIR / "nfl_salary_by_team_2024_complete.csv"
    )
    standings_df = pd.read_csv(DATA_DIR / "nfl_standings_2024.csv")
    
    # Enrich each ranking
    rank_lr_enriched = enrich_rankings_with_external_data(
        rank_lr, salary_df, standings_df, TEAM_NAME_MAP
    )
    rank_knn_enriched = enrich_rankings_with_external_data(
        rank_knn, salary_df, standings_df, TEAM_NAME_MAP
    )
    rank_rf_enriched = enrich_rankings_with_external_data(
        rank_rf, salary_df, standings_df, TEAM_NAME_MAP
    )
    rank_stack_enriched = enrich_rankings_with_external_data(
        rank_stack, salary_df, standings_df, TEAM_NAME_MAP
    )
    
    # Combine all rankings
    all_rankings = pd.concat([
        rank_lr_enriched, rank_knn_enriched,
        rank_rf_enriched, rank_stack_enriched
    ], ignore_index=True)
    
    # -------------------------------------------------------------------------
    # 6. FEATURE IMPORTANCE ANALYSIS
    # -------------------------------------------------------------------------
    
    lr_importance = compute_permutation_importance(
        lr_model, X_test, y_test, feature_cols, "Logistic Regression"
    )
    knn_importance = compute_permutation_importance(
        knn_model, X_test, y_test, feature_cols, "KNN"
    )
    rf_importance = compute_permutation_importance(
        rf_model, X_test, y_test, feature_cols, "Random Forest"
    )
    
    print("\n" + "=" * 60)
    print("TOP 15 FEATURES BY PERMUTATION IMPORTANCE")
    print("=" * 60)
    
    print("\nLogistic Regression:")
    print(lr_importance.head(15).to_string(index=False))

# Guardar CSVs (DENTRO de la función main)
    rank_lr.to_csv(OUTPUT_DIR / "rankings_lr.csv", index=False)
    rank_knn.to_csv(OUTPUT_DIR / "rankings_knn.csv", index=False)
    rank_rf.to_csv(OUTPUT_DIR / "rankings_rf.csv", index=False)
    rank_stack.to_csv(OUTPUT_DIR / "rankings_stack.csv", index=False)
    all_rankings.to_csv(OUTPUT_DIR / "all_rankings_enriched.csv", index=False)
    
    print("\n✅ CSVs SAVED SUCCESSFULLY")


📂 Proyecto: /Users/mariajosezuniga/Desktop/OIM_7502_classwork/AdvProgramming_Final
📊 Datos: /Users/mariajosezuniga/Desktop/OIM_7502_classwork/AdvProgramming_Final/data
💾 Output: /Users/mariajosezuniga/Desktop/OIM_7502_classwork/AdvProgramming_Final/data


In [5]:
main ()



NFL GAME PREDICTION AND TEAM RANKINGS ANALYSIS
LOADING DATA
Columns: 113 | Rows: 1708
Seasons available: <StringArray>
['2022', '2023', '2024']
Length: 3, dtype: string
Season types: <StringArray>
['REG', 'POST']
Length: 2, dtype: string

CREATING TEAM PERSPECTIVES
Dataset expanded to 3416 rows (2 per game)
Average turnovers per game: 1.20

CREATING HISTORICAL FEATURES
Created 16 historical average features

ADDING OPPONENT FEATURES
Merging 17 opponent features...

CREATING RELATIVE FEATURES
Created 16 relative features plus is_home and is_playoff

SPLITTING TRAIN/TEST DATA
Training samples: 4344 (2022, 2023)
Test samples: 2176 (2024)

PREPARING FEATURES
Features used: 20
Sample features: ['rel_passing_yards_team_avg', 'rel_passing_tds_team_avg', 'rel_passing_interceptions_team_avg', 'rel_passing_epa_team_avg', 'rel_rushing_yards_team_avg']

TRAINING LOGISTIC REGRESSION
Cross-validation accuracy: 0.631 (+/- 0.024)
Test accuracy: 0.729

Classification Report:
              precision   